### RAG-Fuison
사용자의 단일 질문을 여러 관점에서 재구성한 뒤, 각 쿼리로 검색한 결과를 Reciprocal Rank Fusion(RRF) 알고리즘으로 병합하는 고급 검색 기법

**RRF 알고리즘**

RRF는 여러 검색 결과 리스트를 하나로 병합하는 알고리즘

In [1]:
from langchain_core.documents import Document

# 기술 뉴스 기사 샘플
documents = [
    Document(
        page_content="OpenAI가 GPT-5를 발표했습니다. 새로운 모델은 이전 버전보다 추론 능력이 크게 향상되었으며, 멀티모달 기능이 강화되었습니다.",
        metadata={"source": "tech_news", "date": "2024-01-15", "topic": "AI"}
    ),
    Document(
        page_content="인공지능 기술의 발전으로 자연어 처리 분야가 빠르게 성장하고 있습니다. 특히 대형 언어 모델(LLM)의 성능 향상이 눈에 띕니다.",
        metadata={"source": "tech_analysis", "date": "2024-01-10", "topic": "AI"}
    ),
    Document(
        page_content="최신 AI 모델들은 코드 생성, 문서 요약, 번역 등 다양한 작업에서 인간 수준의 성능을 보이고 있습니다.",
        metadata={"source": "tech_review", "date": "2024-01-12", "topic": "AI"}
    ),
    Document(
        page_content="GPT 시리즈의 발전 과정을 살펴보면, GPT-3에서 GPT-4로 넘어오면서 논리적 추론과 맥락 이해 능력이 크게 개선되었습니다.",
        metadata={"source": "tech_history", "date": "2024-01-08", "topic": "AI"}
    ),
    Document(
        page_content="기업들이 AI 도입을 가속화하고 있습니다. 특히 고객 서비스, 데이터 분석, 콘텐츠 생성 분야에서 AI 활용이 두드러집니다.",
        metadata={"source": "business_news", "date": "2024-01-14", "topic": "Business"}
    ),
    Document(
        page_content="AI 규제에 대한 논의가 활발합니다. EU는 AI Act를 통해 고위험 AI 시스템에 대한 규제 프레임워크를 마련했습니다.",
        metadata={"source": "policy_news", "date": "2024-01-13", "topic": "Policy"}
    ),
]


In [2]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
vectorstore = FAISS.from_documents(documents, embeddings)
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

e:\Portfolio\Study\hello-langchain\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from langchain_classic.retrievers.multi_query import MultiQueryRetriever
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# MultiQueryRetriever 생성
multi_query_retriever = MultiQueryRetriever.from_llm(
    retriever=retriever,
    llm=llm
)

# 검색 실행
question = "최신 GPT 모델의 성능은 어떤가요?"
docs = multi_query_retriever.invoke(question)

print(f"검색된 문서 수: {len(docs)}")
for i, doc in enumerate(docs, 1):
    print(f"\n[{i}] {doc.page_content[:80]}...")

검색된 문서 수: 3

[1] OpenAI가 GPT-5를 발표했습니다. 새로운 모델은 이전 버전보다 추론 능력이 크게 향상되었으며, 멀티모달 기능이 강화되었습니다....

[2] GPT 시리즈의 발전 과정을 살펴보면, GPT-3에서 GPT-4로 넘어오면서 논리적 추론과 맥락 이해 능력이 크게 개선되었습니다....

[3] 최신 AI 모델들은 코드 생성, 문서 요약, 번역 등 다양한 작업에서 인간 수준의 성능을 보이고 있습니다....


**커스텀 RRF 구현**

- `MultiQueryRetriever`는 내부적으로 결과를 단순 병합합니다. RRF 알고리즘을 직접 구현하면 더 정교한 결과 융합이 가능합니다.

In [4]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from typing import List
from collections import defaultdict

# 1. 다중 쿼리 생성 프롬프트
query_gen_prompt = ChatPromptTemplate.from_template("""
당신은 검색 쿼리를 생성하는 AI 어시스턴트입니다.
주어진 질문에 대해 서로 다른 관점에서 3개의 검색 쿼리를 생성하세요.
각 쿼리는 원본 질문과 같은 정보를 찾되, 다른 키워드나 표현을 사용해야 합니다.

원본 질문: {question}

검색 쿼리들 (한 줄에 하나씩):
""")

# 2. 쿼리 생성 체인
query_gen_chain = query_gen_prompt | llm | StrOutputParser()

# 3. RRF 알고리즘 구현
def reciprocal_rank_fusion(
        results_list: List[List[Document]],
        k: int = 60
) -> List[Document]:
    """
    여러 검색 결과 리스트를 RRF로 병합합니다.

    Args:
        results_list: 각 쿼리별 검색 결과 리스트들
        k: 순위 완화 상수 (기본값 60)

    Returns:
        RRF 점수로 정렬된 문서 리스트
    """

    # 문서별 RRF 점수 계산
    fused_scores = defaultdict(float)
    doc_map = {}  # page_content -> Document 매핑

    for results in results_list:
        for rank, doc in enumerate(results, start=1):
            doc_key = doc.page_content
            fused_scores[doc_key] += 1 / (k + rank)
            doc_map[doc_key] = doc
    
    # 점수 기준 내림차순 정렬
    sorted_docs = sorted(
        fused_scores.items(),
        key=lambda x: x[1],
        reverse=True
    )

    # Document 객체로 변환하여 반환
    return [doc_map[content] for content, score in sorted_docs]

In [5]:
def rag_fusion_retrieve(question: str, top_k: int = 5) -> List[Document]:
    """RAG-Fusion 검색을 수행합니다."""

    # 1. 다중 쿼리 생성
    queries_text = query_gen_chain.invoke({"question": question})
    queries = [q.strip() for q in queries_text.strip().split('\n') if q.strip()]

    # 원본 질문도 포함
    all_queries = [question] + queries

    print(f"생성된 쿼리 ({len(all_queries)}개):")
    for i, q in enumerate(all_queries):
        print(f"  {i+1}. {q}")

    # 2. 각 쿼리로 검색 수행
    all_results = []
    for query in all_queries:
        results = retriever.invoke(query)
        all_results.append(results)

    # 3. RRF로 결과 융합
    fused_docs = reciprocal_rank_fusion(all_results)

    return fused_docs[:top_k]

# 실행
question = "GPT의 발전 과정과 현재 성능"
fused_results = rag_fusion_retrieve(question, top_k=4)

print(f"\n=== RAG-Fusion 결과 (상위 {len(fused_results)}개) ===")
for i, doc in enumerate(fused_results, 1):
    print(f"\n[{i}] {doc.page_content}")


생성된 쿼리 (4개):
  1. GPT의 발전 과정과 현재 성능
  2. 1. GPT의 역사와 현재 기술적 능력 분석
  3. 2. GPT 모델의 진화 과정 및 최신 성능 평가
  4. 3. GPT 발전 이력과 현재의 기능적 특징

=== RAG-Fusion 결과 (상위 4개) ===

[1] GPT 시리즈의 발전 과정을 살펴보면, GPT-3에서 GPT-4로 넘어오면서 논리적 추론과 맥락 이해 능력이 크게 개선되었습니다.

[2] OpenAI가 GPT-5를 발표했습니다. 새로운 모델은 이전 버전보다 추론 능력이 크게 향상되었으며, 멀티모달 기능이 강화되었습니다.

[3] 최신 AI 모델들은 코드 생성, 문서 요약, 번역 등 다양한 작업에서 인간 수준의 성능을 보이고 있습니다.

[4] 인공지능 기술의 발전으로 자연어 처리 분야가 빠르게 성장하고 있습니다. 특히 대형 언어 모델(LLM)의 성능 향상이 눈에 띕니다.


In [6]:
from langchain_core.runnables import RunnablePassthrough

# RAG 프롬프트
rag_prompt = ChatPromptTemplate.from_template("""
다음 컨텍스트를 기반으로 질문에 답변하세요.

컨텍스트:
{context}

질문: {question}

답변:
""")

def format_docs(docs: List[Document]) -> str:
    return "\n\n".join(doc.page_content for doc in docs)

# RAG-Fusion 기반 체인
rag_fusion_chain = (
    {
        "context": lambda x: format_docs(rag_fusion_retrieve(x["question"])),
        "question": lambda x: x["question"]
    }
    | rag_prompt
    | llm
    | StrOutputParser()
)

# 실행
response = rag_fusion_chain.invoke({
    "question": "GPT 모델이 어떻게 발전해왔고 현재 어떤 수준인가요?"
})
print(response)


생성된 쿼리 (4개):
  1. GPT 모델이 어떻게 발전해왔고 현재 어떤 수준인가요?
  2. 1. GPT 모델의 발전 과정과 현재 기술 수준에 대한 개요는 무엇인가요?
  3. 2. GPT 기술의 역사와 현재의 성능은 어떻게 변화해왔나요?
  4. 3. 최신 GPT 모델의 발전 이력과 현재의 기능적 특징은 무엇인가요?
GPT 모델은 초기 버전인 GPT-1에서 시작하여, GPT-2, GPT-3, 그리고 최근의 GPT-4와 GPT-5로 발전해왔습니다. 각 버전은 이전 모델에 비해 성능이 크게 향상되었으며, 특히 GPT-3에서 GPT-4로 넘어오면서 논리적 추론과 맥락 이해 능력이 크게 개선되었습니다. 

최근 발표된 GPT-5는 이러한 발전을 이어받아 추론 능력이 더욱 향상되었고, 멀티모달 기능이 강화되었습니다. 현재의 AI 모델들은 코드 생성, 문서 요약, 번역 등 다양한 작업에서 인간 수준의 성능을 보이고 있으며, 이는 자연어 처리 분야의 빠른 성장과 대형 언어 모델(LLM)의 성능 향상에 기인하고 있습니다. 이러한 발전 덕분에 인공지능 기술은 다양한 분야에서 실질적인 응용이 가능해지고 있습니다.


In [7]:
def compare_retrieval(question: str):
    """단일 쿼리와 RAG-Fusion 결과를 비교합니다."""

    # 단일 쿼리 검색
    single_results = retriever.invoke(question)

    # RAG-Fusion 검색
    fusion_results = rag_fusion_retrieve(question, top_k=len(single_results))

    print(f"질문: {question}\n")

    print("=== 단일 쿼리 결과 ===")
    for i, doc in enumerate(single_results, 1):
        print(f"[{i}] {doc.page_content[:60]}...")

    print("\n=== RAG-Fusion 결과 ===")
    for i, doc in enumerate(fusion_results, 1):
        print(f"[{i}] {doc.page_content[:60]}...")

    # 차이점 분석
    single_contents = {doc.page_content for doc in single_results}
    fusion_contents = {doc.page_content for doc in fusion_results}

    only_in_fusion = fusion_contents - single_contents
    print(f"\n추가로 찾은 문서: {len(only_in_fusion)}개")

compare_retrieval("AI 기술의 최신 동향")


생성된 쿼리 (4개):
  1. AI 기술의 최신 동향
  2. 1. 최신 인공지능 기술 발전 현황
  3. 2. 최근 AI 트렌드 및 혁신
  4. 3. 인공지능 분야의 최신 연구 및 동향
질문: AI 기술의 최신 동향

=== 단일 쿼리 결과 ===
[1] 기업들이 AI 도입을 가속화하고 있습니다. 특히 고객 서비스, 데이터 분석, 콘텐츠 생성 분야에서 AI 활용...
[2] 최신 AI 모델들은 코드 생성, 문서 요약, 번역 등 다양한 작업에서 인간 수준의 성능을 보이고 있습니다....
[3] AI 규제에 대한 논의가 활발합니다. EU는 AI Act를 통해 고위험 AI 시스템에 대한 규제 프레임워크를...

=== RAG-Fusion 결과 ===
[1] 기업들이 AI 도입을 가속화하고 있습니다. 특히 고객 서비스, 데이터 분석, 콘텐츠 생성 분야에서 AI 활용...
[2] 최신 AI 모델들은 코드 생성, 문서 요약, 번역 등 다양한 작업에서 인간 수준의 성능을 보이고 있습니다....
[3] 인공지능 기술의 발전으로 자연어 처리 분야가 빠르게 성장하고 있습니다. 특히 대형 언어 모델(LLM)의 성능...

추가로 찾은 문서: 1개
